In [ ]:
import pandas as pd
import json
import os
import glob
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import re
import base64
import numpy as np

#! #######################     PATHS    #############################################################
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE_NOVO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Estoques"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"
CAMINHO_DO_LOGO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\images\logo_inv.png" 
PATH_FERIADOS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\feriados_nacionais.csv"

NOME_DO_FUNDO = "FIDC FCT CONSIGNADO II"
NOME_ARQUIVO_SAIDA = "relatorio_fluxo_caixa_final.html"
NOME_ARQUIVO_SAIDA_SIMPLES = "relatorio_fluxo_caixa_simples.html"
#! #################################################################################################

#? AUX  <--------------------------------------------------------------

def adicionar_dias_uteis(data_inicial, dias_a_adicionar, feriados):
    """
    obs: para d+0, retrna a prpra data se for dia util, ou o prxmo dia util.
    """
    data_atual = data_inicial
    dias_uteis_contados = 0
    
    # se o numro de dias a adicnr for 0, o primr passo eh encntr o dia util atul ou o prxmo
    if dias_a_adicionar == 0:
        while data_atual.weekday() >= 5 or data_atual.date() in feriados:
            data_atual += timedelta(days=1)
        return data_atual

    # adicn os dias utes um por um
    while dias_uteis_contados < dias_a_adicionar:
        data_atual += timedelta(days=1)
        if data_atual.weekday() < 5 and data_atual.date() not in feriados:
            dias_uteis_contados += 1
            
    return data_atual

def encontrar_n_dia_util(ano, mes, n, feriados):
    """encntr o n-ehsmo dia util de um mes."""
    dia_atual = date(ano, mes, 1)
    dias_uteis_encontrados = 0
    while dias_uteis_encontrados < n:
        if dia_atual.weekday() < 5 and dia_atual not in feriados:
            dias_uteis_encontrados += 1
        if dias_uteis_encontrados < n:
            dia_atual += timedelta(days=1)
    return dia_atual

def codificar_imagem_base64(caminho_imagem):
    """convrt uma imgm para bas64 para embtr no html."""
    try:
        with open(caminho_imagem, "rb") as image_file: return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError: return "iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR42mNkYAAAAAYAAjCB0C8AAAAASUVORK5CYII="

# FUNCOES De CARREGAR DADOS: 

def encontrar_e_carregar_dados_sincronizados(path_carteiras, path_despesas):
    """encntr e carrga o melhr par de arqvs de cartear e despess."""
    print("Sincronizando arquivos de Carteira e Despesas...")
    mapa_carteiras, mapa_despesas = {}, {}
    for arquivo in glob.glob(os.path.join(path_carteiras, "*.json")):
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                dados = json.load(f)
                data_obj = datetime.strptime(dados[0]['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
                mapa_carteiras[data_obj] = arquivo
        except Exception: continue
    for arquivo in glob.glob(os.path.join(path_despesas, '**/Despesas_Consolidadas.xlsx'), recursive=True):
        try:
            df_temp = pd.read_excel(arquivo, header=6)
            df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce', dayfirst=True)
            data_obj = df_temp['Data'].max()
            if pd.notna(data_obj): mapa_despesas[data_obj] = arquivo
        except Exception: continue
    if not mapa_carteiras or not mapa_despesas: raise FileNotFoundError("Arquivos de carteira ou despesas não encontrados.")
    
    path_carteira_final, path_despesa_final, data_carteira_final, data_despesa_final = None, None, None, None
    for data_c in sorted(mapa_carteiras.keys(), reverse=True):
        datas_despesas_candidatas = [data_d for data_d in mapa_despesas.keys() if data_d <= data_c]
        if datas_despesas_candidatas:
            data_d_final = max(datas_despesas_candidatas)
            path_carteira_final, path_despesa_final = mapa_carteiras[data_c], mapa_despesas[data_d_final]
            data_carteira_final, data_despesa_final = data_c, data_d_final
            break
    if not path_carteira_final: raise FileNotFoundError("Nenhuma combinação válida de arquivos encontrada.")

    print(f"combinação sincronizada:")
    print(f"  - file de Carteira: {os.path.basename(path_carteira_final)} (Data: {data_carteira_final.strftime('%d/%m/%Y')})")
    print(f"  - Base de Despesas...: Relatório gerado em {data_despesa_final.strftime('%d/%m/%Y')}")
    
    with open(path_carteira_final, 'r', encoding='utf-8') as f:
        dados_carteira = json.load(f)[0]
    
    df_despesas_base = pd.read_excel(path_despesa_final, header=6)
    df_base = df_despesas_base.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor', 'Data Pagamento': 'Data Pagamento'})
    df_base['Valor'] = -pd.to_numeric(df_base['Valor'], errors='coerce').abs()
    df_base['Data Pagamento'] = pd.to_datetime(df_base['Data Pagamento'], errors='coerce', dayfirst=True)
    df_base = df_base[['Categoria', 'Valor', 'Data Pagamento']].dropna()
    
    despesas_reais_futuras = df_base[df_base['Data Pagamento'] >= data_carteira_final]
    ultimo_mes_real = despesas_reais_futuras['Data Pagamento'].max().replace(day=1) if not despesas_reais_futuras.empty else None
    mes_inicial_projecao = (ultimo_mes_real + relativedelta(months=1)) if ultimo_mes_real else (data_carteira_final.replace(day=1) + relativedelta(months=1))

    feriados = set()
    try:
        feriados = set(pd.to_datetime(pd.read_csv(PATH_FERIADOS)['Data']).dt.date)
    except FileNotFoundError:
        print(f"AVISO: Arquivo de feriados não encontrado. Coloque o arquivo aqui! ")
        
    despesas_projetadas = []
    df_molde = df_base.drop_duplicates(subset=['Categoria'])
    for i in range(12): # Proj 12 meses
        mes_atual = mes_inicial_projecao + relativedelta(months=i)
        data_pagamento_proj = encontrar_n_dia_util(mes_atual.year, mes_atual.month, 4, feriados)
        data_pagamento_proj_dt = datetime.combine(data_pagamento_proj, datetime.min.time())
        if data_pagamento_proj_dt >= data_carteira_final:
            for _, row in df_molde.iterrows():
                despesas_projetadas.append({'Data Pagamento': data_pagamento_proj_dt, 'Categoria': row['Categoria'], 'Valor': row['Valor']})
    
    df_despesas = pd.concat([despesas_reais_futuras, pd.DataFrame(despesas_projetadas)], ignore_index=True)
    return dados_carteira, data_carteira_final, df_despesas, feriados

def gerar_matriz_cenarios_pdd(df_fluxos, data_base, feriados):
    """
    gera uma matrz de estrss, mostrn o saldo finl em cada prazo d+x
    para difrnt cenrs de pdd.
    """
    print("\nGerando matriz de cenários de PDD...")
    prazos = [0, 1, 5, 10, 21, 63, 365] # Usando os mesmos prazos da tabela principal
    cenarios_pdd = [0.0, 0.20, 0.40, 0.60, 0.80, 1.00]
    resultados = []

    for pdd in cenarios_pdd:
        linha_cenario = {'Cenário PDD': f'{pdd:.0%}'}
        
        # cria uma copia dos flxs para modfcr neste cenro espcfc
        df_fluxos_cenario = df_fluxos.copy()
        df_fluxos_cenario.loc[df_fluxos_cenario['categoria'] == 'Pagam Estoq. Bruto', 'valor'] *= (1 - pdd)
        
        # calcla o saldo de caxa acumld para cada przo d+x
        for dias in prazos:
            data_limite = adicionar_dias_uteis(data_base, dias, feriados)
            saldo_final = df_fluxos_cenario[df_fluxos_cenario['data'] <= data_limite]['valor'].sum()
            linha_cenario[f'D+{dias}'] = saldo_final
            
        resultados.append(linha_cenario)

    # cria o df final e defne o cenario como indce
    df_matriz = pd.DataFrame(resultados)
    return df_matriz.set_index('Cenário PDD')

def processar_dados_estoque_novo(caminho_pasta):
    """le o conjnt de arqvs de estque mais recnte da pasta."""
    print("\nCarregando dados do novo estoque...")
    todos_arquivos = glob.glob(os.path.join(caminho_pasta, '**/*.csv'), recursive=True)
    if not todos_arquivos: return pd.DataFrame()
    data_recente = None
    regex_data = re.compile(r'(\d{2}\.\d{2}\.\d{2})')
    for arquivo in todos_arquivos:
        match = regex_data.search(os.path.basename(arquivo))
        if match:
            try:
                data_obj = datetime.strptime(match.group(1), '%d.%m.%y')
                if data_recente is None or data_obj > data_recente: data_recente = data_obj
            except ValueError: continue
    if data_recente is None: return pd.DataFrame()
    data_recente_str = data_recente.strftime('%d.%m.%y')
    print(f"Usando o conjunto de estoque da data: {data_recente_str}")
    arquivos_para_ler = [f for f in todos_arquivos if data_recente_str in os.path.basename(f)]
    lista_dfs_partes = []
    for arquivo in arquivos_para_ler:
        try:
            df_parte = pd.read_csv(arquivo, sep=';', decimal=',', encoding='utf-8', on_bad_lines='warn')
            lista_dfs_partes.append(df_parte)
        except Exception as e: print(f"AVISO: Falha ao ler o arquivo '{os.path.basename(arquivo)}'. Erro: {e}")
    if not lista_dfs_partes: return pd.DataFrame()
    df_consolidado = pd.concat(lista_dfs_partes, ignore_index=True)
    
    df_consolidado.rename(columns={
        'DataVencimento': 'Data Vencimento', 
        'ValorNominal': 'Valor Nominal', 
        'PDDTotal': 'PDDTotal', 
        'NumeroTitulo': 'NumeroTitulo', 
        'DataGeracao': 'DataGeracao',
        'ValorPresente': 'Valor Presente' #! obs <<
    }, inplace=True)
    
    # convrt os tips de dads
    colunas_data = ['Data Vencimento', 'DataGeracao']
    for col in colunas_data:
        if col in df_consolidado.columns:
            df_consolidado[col] = pd.to_datetime(df_consolidado[col], dayfirst=True, errors='coerce')
    
    colunas_num = ['Valor Nominal', 'PDDTotal', 'Valor Presente']
    for col in colunas_num:
        if col in df_consolidado.columns:
            df_consolidado[col] = pd.to_numeric(df_consolidado[col], errors='coerce')

    if 'PDDTotal' in df_consolidado.columns and 'Valor Presente' in df_consolidado.columns and 'Valor Nominal' in df_consolidado.columns:
        print("Corrigindo a PDD para base nominal...")  # pq a pdd é em valo real, agora uso a nominal
        
        # nulos com 0
        vp = df_consolidado['Valor Presente'].fillna(0)
        pdd_vp = df_consolidado['PDDTotal'].fillna(0)
        
        # proporção da PDD sobre o VP
        proporcao_pdd = np.divide(pdd_vp, vp, out=np.zeros_like(pdd_vp, dtype=float), where=vp!=0) # trato div por 0
        
        # aplico a proporção
        df_consolidado['PDDTotal'] = proporcao_pdd * df_consolidado['Valor Nominal'].fillna(0)
            

    print(f"Dados do novo estoque consolidados: {len(df_consolidado)} linhas.")
    return df_consolidado


def preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas_datadas):
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    lista_de_fluxos_df = []

    # parte da Carteira JSON
    disponibilidade = next((a.get('Disponibilidade') for a in dados_carteira_recente['ativos'] if a.get('Disponibilidade')), None)
    if disponibilidade: lista_de_fluxos_df.append(pd.DataFrame([{'data': data_base, 'categoria': 'Conta Corr.', 'valor': disponibilidade['ativos'][0]['saldo'], 'descricao': 'Saldo Inicial'}]))
    cotas = next((a.get('Cotas') for a in dados_carteira_recente['ativos'] if a.get('Cotas')), None)
    if cotas:
        df_fundos = pd.DataFrame(cotas['ativos']); df_fundos['data'], df_fundos['categoria'], df_fundos['valor'], df_fundos['descricao'] = data_base, 'Fundo inv.', df_fundos['valorBruto'], "Resgate " + df_fundos['titulo']; lista_de_fluxos_df.append(df_fundos[['data', 'categoria', 'valor', 'descricao']])
    renda_fixa = next((a.get('RendaFixa') for a in dados_carteira_recente['ativos'] if a.get('RendaFixa')), None)
    if renda_fixa:
        df_rf = pd.DataFrame(renda_fixa['ativos']); df_rf['data'], df_rf['categoria'], df_rf['valor'], df_rf['descricao'] = pd.to_datetime(df_rf['dataVencimento']), 'Títulos Renda Fixa', df_rf['mercadoAtual'], "Venc. " + df_rf['codigoCustodia']; lista_de_fluxos_df.append(df_rf[['data', 'categoria', 'valor', 'descricao']])

    # Estoque
    if not df_estoque.empty:
        estoque_futuro = df_estoque[df_estoque['Data Vencimento'] >= data_base].dropna(subset=['Data Vencimento', 'Valor Nominal', 'PDDTotal']).copy()
        print(f"\n{len(estoque_futuro)} parcelas do estoque com vencimento futuro foram processadas.")
        df_estoque_bruto = pd.DataFrame({'data': estoque_futuro['Data Vencimento'], 'categoria': 'Pagam Estoq. Bruto', 'valor': estoque_futuro['Valor Nominal'], 'descricao': "Título Nº " + estoque_futuro['NumeroTitulo'].astype(str)})
        lista_de_fluxos_df.append(df_estoque_bruto)
        estoque_com_pdd = estoque_futuro[estoque_futuro['PDDTotal'] > 0].copy()
        if not estoque_com_pdd.empty:
            df_estoque_pdd = pd.DataFrame({'data': estoque_com_pdd['Data Vencimento'], 'categoria': 'PDD Estoque', 'valor': -estoque_com_pdd['PDDTotal'], 'descricao': "Provisão PDD Título Nº " + estoque_com_pdd['NumeroTitulo'].astype(str)})
            lista_de_fluxos_df.append(df_estoque_pdd)

    # Despesas
    if not df_despesas_datadas.empty:
        despesas_futuras = df_despesas_datadas[df_despesas_datadas['Data Pagamento'] >= data_base].copy()
        despesas_futuras['categoria'], despesas_futuras['descricao'] = 'Despesas', despesas_futuras['Categoria']
        df_despesas_final = despesas_futuras.rename(columns={'Data Pagamento': 'data', 'Valor': 'valor'})
        lista_de_fluxos_df.append(df_despesas_final[['data', 'categoria', 'valor', 'descricao']])

    # Amortização
    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        valor_amortizacao = (carteira_senior['pl'] / 36) * -1
        datas_amortizacao, descricoes_amortizacao = [], []
        data_base_amort = datetime(2026, 1, 16)
        for i in range(36):
            data_pagamento = data_base_amort + relativedelta(months=i)
            while data_pagamento.weekday() >= 5: data_pagamento += pd.Timedelta(days=1)
            datas_amortizacao.append(data_pagamento); descricoes_amortizacao.append(f"Parcela {i+1}/36")
        df_amort = pd.DataFrame({'data': datas_amortizacao, 'categoria': 'Amort./ Resgat. Cota', 'valor': valor_amortizacao, 'descricao': descricoes_amortizacao})
        lista_de_fluxos_df.append(df_amort)
        
    if not lista_de_fluxos_df: return pd.DataFrame()
    return pd.concat(lista_de_fluxos_df, ignore_index=True)

def gerar_relatorio_d_mais(df_fluxos, data_base, feriados):
    prazos = [0, 1, 5, 10, 21, 42, 126, 252]
    
    colunas_calculo = ['Conta Corr.', 'Fundo inv.', 'Títulos Renda Fixa', 'Pagam Estoq. Bruto', 'PDD Estoque', 'Despesas', 'Amort./ Resgat. Cota']
    
    linhas_relatorio = []
    for dias in prazos:
        data_limite = adicionar_dias_uteis(data_base, dias, feriados)
        fluxos_ate_data = df_fluxos[df_fluxos['data'] <= data_limite]
        saldos_acumulados = fluxos_ate_data.groupby('categoria')['valor'].sum()
        
        linha = {
            'Prazo': f"D+{dias}",
            'Data': data_limite # <-- ADICIONADO
        }
        for col in colunas_calculo:
            linha[col] = saldos_acumulados.get(col, 0.0)
        
        linhas_relatorio.append(linha)
            
    df_relatorio = pd.DataFrame(linhas_relatorio).set_index('Prazo')
    
    # --- Cálculos das colunas ---
    df_relatorio['Pagam. Estoq Liq Pdd'] = df_relatorio['Pagam Estoq. Bruto'] + df_relatorio['PDD Estoque']
    df_relatorio['Disponibilidades'] = df_relatorio['Conta Corr.'] + df_relatorio['Fundo inv.'] + df_relatorio['Títulos Renda Fixa'] + df_relatorio['Pagam. Estoq Liq Pdd']
    df_relatorio['Necessidades'] = df_relatorio['Despesas'] + df_relatorio['Amort./ Resgat. Cota']
    df_relatorio['Caixa Projetado'] = df_relatorio['Disponibilidades'] + df_relatorio['Necessidades']
    
    epsilon = 1e-9
    df_relatorio['Índice de Liquidez'] = df_relatorio['Disponibilidades'] / (-df_relatorio['Necessidades'] + epsilon)
    
    gap_a_cobrir = -df_relatorio['Necessidades'] - (df_relatorio['Conta Corr.'] + df_relatorio['Fundo inv.'] + df_relatorio['Títulos Renda Fixa'])
    gap_a_cobrir[gap_a_cobrir < 0] = 0
    df_relatorio['Inadimplência Break-Even'] = 1 - (gap_a_cobrir / (df_relatorio['Pagam Estoq. Bruto'] + epsilon))
    
    
    # Escala (/1000)
    colunas_monetarias = [
        'Caixa Projetado', 'Disponibilidades', 'Necessidades', 'Conta Corr.', 
        'Fundo inv.', 'Títulos Renda Fixa', 'Pagam Estoq. Bruto', 
        'Pagam. Estoq Liq Pdd', 'Despesas', 'Amort./ Resgat. Cota'
    ]
    for col in colunas_monetarias:
        if col in df_relatorio.columns:
            df_relatorio[col] = df_relatorio[col] / 1000

    # ORDEM das Cols
    ordem_final = [
        'Data', 
        'Caixa Projetado', 'Índice de Liquidez', 'Inadimplência Break-Even', 
        'Disponibilidades', 'Necessidades', 'Conta Corr.', 'Fundo inv.', 
        'Títulos Renda Fixa', 'Pagam Estoq. Bruto', 'Pagam. Estoq Liq Pdd', 
        'Despesas', 'Amort./ Resgat. Cota'
    ]
    df_relatorio = df_relatorio[[col for col in ordem_final if col in df_relatorio.columns]]

    # 3. Nomes com quebra de linha
    novos_nomes = {
        'Data': "Data", 
        'Caixa Projetado': "Saldo Final<br><span class='unit'>('000 R$)</span>",
        'Índice de Liquidez': "Índice Liq.<br><span class='unit'>(x)</span>",
        'Inadimplência Break-Even': "Inadimplência B.E.<br><span class='unit'>(%)</span>",
        'Disponibilidades': "Total Disp.<br><span class='unit'>('000 R$)</span>",
        'Necessidades': "Total Nec.<br><span class='unit'>('000 R$)</span>",
        'Conta Corr.': "C/C<br><span class='unit'>('000 R$)</span>",
        'Fundo inv.': "Fundos<br><span class='unit'>('000 R$)</span>",
        'Títulos Renda Fixa': "Renda Fixa<br><span class='unit'>('000 R$)</span>",
        'Pagam Estoq. Bruto': "Estoque Bruto<br><span class='unit'>('000 R$)</span>",
        'Pagam. Estoq Liq Pdd': "Estoque Líq.<br><span class='unit'>('000 R$)</span>",
        'Despesas': "Despesas<br><span class='unit'>('000 R$)</span>",
        'Amort./ Resgat. Cota': "Amortização<br><span class='unit'>('000 R$)</span>"
    }
    df_relatorio.rename(columns=novos_nomes, inplace=True)

    # -------- parte de formatar os numeros  ------------------
    df_formatado = df_relatorio.copy()
    
    # nomes de colunas que eu vou dar um format especifico, como
    col_data = "Data" 
    col_indice_liq = "Índice Liq.<br><span class='unit'>(x)</span>"
    col_inadimplencia = "Inadimplência B.E.<br><span class='unit'>(%)</span>"

    def formatar_liquidez(valor):
        if valor >= 20: return ">20"
        return f"{valor:,.1f}"

    def formatar_breakeven(valor):
        if valor >= 0.9999:  # quase 100%
            return "*"
        return f"{(valor * 100):.1f}%"

    for col in df_formatado.columns:  # vou passar formatando cada coluna
        if col == col_data: # 
             df_formatado[col] = pd.to_datetime(df_formatado[col]).dt.strftime('%d/%m/%Y') # 
        elif col == col_indice_liq:
            df_formatado[col] = df_formatado[col].apply(formatar_liquidez)
        elif col == col_inadimplencia:
            df_formatado[col] = df_formatado[col].apply(formatar_breakeven)
        # Formato o que sobra- as monetarias
        elif pd.api.types.is_numeric_dtype(df_formatado[col]):
            df_formatado[col] = df_formatado[col].apply('{:,.0f}'.format)
            
    return df_formatado
    
def gerar_relatorio_html_final(df_summary_formatted, df_details, df_matriz_cenarios, nome_fundo, data_ref, caminho_logo, arquivo_saida):
    logo_base64 = codificar_imagem_base64(caminho_logo)
    
    # pop-up pra detalhar os dados
    df_details_leve = df_details[~df_details['categoria'].str.contains("Estoq", na=False)].copy()
    if not df_details_leve.empty:
        df_details_leve['data'] = pd.to_datetime(df_details_leve['data'], errors='coerce')
        df_details_leve.dropna(subset=['data'], inplace=True)
        df_details_leve['data'] = df_details_leve['data'].dt.strftime('%d/%m/%Y')
    dados_detalhados_json = df_details_leve.to_json(orient='records', force_ascii=False)

    mapa_reverso_nomes = {
        "C/C('000 R$)": 'Conta Corr.', "Fundos('000 R$)": 'Fundo inv.',
        "Renda Fixa('000 R$)": 'Títulos Renda Fixa', "Despesas('000 R$)": 'Despesas',
        "Amortização('000 R$)": 'Amort./ Resgat. Cota'
    }

    # builda a HTML principal
    tabela_html = '<table id="relatorioTabela" class="display compact"><thead><tr><th>Prazo</th>'
    for col in df_summary_formatted.columns: tabela_html += f'<th>{col}</th>' 
    tabela_html += '</tr></thead><tbody>'
    for prazo, row in df_summary_formatted.iterrows():
        tabela_html += f'<tr><th>{prazo}</th>'
        row_com_atributos = row.rename(lambda x: re.sub('<.*?>', '', x).strip())
        for col_stripped, value in row_com_atributos.items():
            categoria_original = mapa_reverso_nomes.get(col_stripped, col_stripped)
            tabela_html += f'<td data-prazo="{prazo}" data-categoria="{categoria_original}">{value}</td>'
    tabela_html += '</tr></tbody></table>'

    #  MATRIZ DE CENÁRIOS >>>>>>>>
    tabela_cenarios_html = ""
    if not df_matriz_cenarios.empty:
        # Formato (div 1000 e formata como int)
        df_matriz_fmt = (df_matriz_cenarios / 1000).applymap('{:,.0f}'.format)
        
        # Converto para HTML  >>>>>>>
        tabela_cenarios_html = "<h2>Matriz de Estresse de Saldo de Caixa ('000 R$)</h2>"
        tabela_cenarios_html += '<table class="display compact"><thead><tr>'
        tabela_cenarios_html += f'<th>{df_matriz_fmt.index.name}</th>'
        for col in df_matriz_fmt.columns: tabela_cenarios_html += f'<th>{col}</th>'
        tabela_cenarios_html += '</tr></thead><tbody>'
        for index, row in df_matriz_fmt.iterrows():
            tabela_cenarios_html += f'<tr><th>{index}</th>'
            for value in row:
                tabela_cenarios_html += f'<td style="text-align:right;">{value}</td>'
            tabela_cenarios_html += '</tr>'
        tabela_cenarios_html += '</tbody></table>'
    # --- FIM DA ATUALIZAÇÃO ---

    html_template = f"""
    <!DOCTYPE html><html lang="pt-BR"><head><meta charset="UTF-8"><title>Relatório de Fluxo de Caixa</title>
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.13.6/css/jquery.dataTables.min.css">
    <style>
        body{{font-family:"Gill Sans MT",Arial,sans-serif;background-color:#FFFFFF;color:#313131;font-size:1.1em;padding:20px}}
        h1{{font-size:2.2em;color:#163f3f;margin:0;line-height:1;}}
        h2{{font-size:1.6em;color:#163f3f;margin-top:40px; border-bottom: 2px solid #e0e0e0; padding-bottom: 5px;}}
        .header-container{{display:flex;align-items:center;margin-bottom:20px}}
        .info-box{{border:1px solid #e0e0e0;background-color:#f5f5f5;padding:15px;margin-bottom:30px}}
        table.dataTable thead th{{background-color:#163f3f!important;color:white!important; text-align:center; vertical-align:middle;}}
        table.dataTable.display tbody tr.odd{{background-color:#f5f5f5}} table.dataTable.display tbody tr.even{{background-color:#e0e0e0}}
        table.dataTable tbody tr:hover{{background-color:#FFD8A7!important}} table.dataTable tbody th{{font-weight:bold;background-color:#0e3030;color:white}}
        #relatorioTabela tbody td{{cursor:default; text-align:right;}}
        #relatorioTabela tbody td:not([data-categoria*="Estoq"]):not([data-categoria*="Disp."]):not([data-categoria*="Nec."]):not([data-categoria*="Caixa"]):not(:empty){{cursor:pointer}}
        .modal{{display:none;position:fixed;z-index:1000;left:0;top:0;width:100%;height:100%;overflow:auto;background-color:rgba(0,0,0,.5)}}
        .modal-content{{background-color:#fefefe;margin:5% auto;padding:20px;border:1px solid #888;width:80%;max-width:900px}}
        .close-button{{color:#aaa;float:right;font-size:28px;font-weight:bold;cursor:pointer}}
        #modal-table-container table{{width:100%;border-collapse:collapse}} #modal-table-container th,#modal-table-container td{{border:1px solid #ddd;padding:8px;text-align:left}}
        #modal-table-container th{{background-color:#f2f2f2}}
        .unit {{ font-weight: normal; font-size: 0.8em; color: #d3d3d3; }}
    </style></head><body>
    <div class="header-container"><img src="data:image/png;base64,{logo_base64}" style="max-height:80px;margin-right:20px;"><h1>Relatório de<br>Fluxo de Caixa</h1></div>
    <div class="info-box"><strong>Fundo:</strong> {nome_fundo}<br><strong>Data do Relatório:</strong> {datetime.now().strftime('%d/%m/%Y')}<br><strong>Data de Referência (D+0):</strong> {data_ref.strftime('%d/%m/%Y')}</div>
    {tabela_html}
    {tabela_cenarios_html} 
    <div id="detailModal" class="modal"><div class="modal-content"><span class="close-button">&times;</span><h2 id="modal-title">Detalhamento</h2><div id="modal-table-container" style="max-height:400px;overflow-y:auto"></div></div></div>
    <script src="https://code.jquery.com/jquery-3.7.0.js"></script><script src="https://cdn.datatables.net/1.13.6/js/jquery.dataTables.min.js"></script>
    <script>var dadosDetalhados={dados_detalhados_json};var dataReferencia=new Date('{data_ref.strftime("%Y-%m-%d")}T00:00:00');</script>
    <script>
    $(document).ready(function(){{$('table.display').DataTable({{"paging":!1,"searching":!1,"info":!1,"ordering":!1}});var a=$('#detailModal');$('#relatorioTabela tbody').on('click','td',function(){{var b=$(this),c=b.data('prazo'),d=b.data('categoria');if(c&&d&&!d.includes("Disp.")&&!d.includes("Nec.")&&!d.includes("Caixa")&&!d.includes("Estoq")&&"0.00"!==b.text().trim()&&"0,00"!==b.text().trim()){{var e=parseInt(c.replace('D+','')),f=new Date(dataReferencia);f.setDate(f.getDate()+e);var g=dadosDetalhados.filter(function(a){{var b=new Date(a.data.split('/').reverse().join('-')+"T00:00:00");return a.categoria===d&&b<=f}}),h="<table><thead><tr><th>Data</th><th>Descrição</th><th style=\\"text-align:right;\\">Valor</th></tr></thead><tbody>",i=0;g.forEach(function(a){{i+=a.valor; var valorFormatado = a.valor.toLocaleString("pt-BR",{{style:"currency",currency:"BRL",minimumFractionDigits:2,maximumFractionDigits:2}}); h+="<tr><td>"+a.data+"</td><td>"+(a.descricao||"")+"</td><td style=\\"text-align:right;\\">"+valorFormatado+"</td></tr>"}}),h+="</tbody><tfoot><tr><th colspan=\\"2\\">Total</th><th style=\\"text-align:right;\\">"+i.toLocaleString("pt-BR",{{style:"currency",currency:"BRL",minimumFractionDigits:2,maximumFractionDigits:2}})+"</th></tr></tfoot></table>",$('#modal-title').text("Detalhamento de: "+d+" (até "+c+")"),$('#modal-table-container').html(h),a.show()}}}}),$('.close-button').on('click',function(){{a.hide()}}),$(window).on('click',function(b){{b.target==a[0]&&a.hide()}})}});
    </script></body></html>"""
    try:
        with open(arquivo_saida, 'w', encoding='utf-8') as f: f.write(html_template)
        print(f"✅ Relatório HTML com design aprimorado gerado com sucesso: '{os.path.abspath(arquivo_saida)}'")
    except Exception as e: print(f"❌ Erro ao salvar o arquivo HTML: {e}")


def gerar_relatorio_html_simples(df_summary_formatted, nome_fundo, data_ref, arquivo_saida):
    
    tabela_html = '<table><thead><tr><th>Prazo</th>'
    for col in df_summary_formatted.columns:
        tabela_html += f'<th>{col}</th>'
    tabela_html += '</tr></thead><tbody>'
    for prazo, row in df_summary_formatted.iterrows():
        tabela_html += f'<tr><th>{prazo}</th>'
        for value in row:
            tabela_html += f'<td>{value}</td>'
        tabela_html += '</tr>'
    tabela_html += '</tbody></table>'
    
    html_template = f"""
    <!DOCTYPE html><html lang="pt-BR"><head><meta charset="UTF-8"><title>Relatório Simplificado de Fluxo de Caixa</title>
    <style>
        body{{font-family:"Gill Sans MT",Arial,sans-serif;background-color:#FFFFFF;color:#313131;padding:20px; font-size:1.1em;}}
        h1{{font-size:2.2em;color:#163f3f;margin:0;line-height:1;}}
        .info-box{{border:1px solid #e0e0e0;background-color:#f5f5f5;padding:15px;margin-bottom:30px;}}
        table{{width: 100%; border-collapse: collapse;}}
        th, td{{border: 1px solid #ddd; padding: 8px; text-align: right;}}
        thead th{{background-color:#163f3f;color:white; text-align:center; vertical-align:middle;}}
        
        /* --- syle zebrinha COM DOIS TONS DE CINZA --- */
        tbody tr:nth-child(odd)  {{ background-color: #f5f5f5; }} /* Cinza mais claro para linhas ímpares */
        tbody tr:nth-child(even) {{ background-color: #e0e0e0; }} /* Cinza um pouco mais escuro para linhas pares */

        tbody th{{font-weight:bold;background-color:#0e3030;color:white; text-align:left;}}
        .unit {{ font-weight: normal; font-size: 0.8em; color: #d3d3d3; }}
    </style></head><body>
    <h1>Relatório Simplificado de<br>Fluxo de Caixa</h1>
    <div class="info-box"><strong>Fundo:</strong> {nome_fundo}<br><strong>Data de Referência (D+0):</strong> {data_ref.strftime('%d/%m/%Y')}</div>
    {tabela_html}
    </body></html>"""
    
    try:
        with open(arquivo_saida, 'w', encoding='utf-8') as f:
            f.write(html_template)
        print(f"✅ Relatório HTML simplificado (com zebrado cinza) gerado: '{os.path.abspath(arquivo_saida)}'")
    except Exception as e:
        print(f"❌ Erro ao salvar o arquivo HTML simplificado: {e}")

        
if __name__ == "__main__":
    try:
        dados_carteira_recente, data_referencia, df_despesas, feriados = encontrar_e_carregar_dados_sincronizados(PATH_CARTEIRAS_HISTORICO, PATH_DESPESAS)
        df_estoque = processar_dados_estoque_novo(PATH_ESTOQUE_NOVO)
        
        print("\n" + "="*50)
        print(" <DEBUG> FONTES DE DADOS UTILIZADAS ")  # isso aqui serve para 
        print(f"1. PONTO DE PARTIDA (D+0): {data_referencia.strftime('%d/%m/%Y')}")
        if not df_estoque.empty:
            data_geracao_estoque = pd.to_datetime(df_estoque['DataGeracao'], errors='coerce').dropna().max()
            if pd.notna(data_geracao_estoque):
                 print(f" ENTRADAS DE CAIXA: Usando conjunto de estoque gerado em {data_geracao_estoque.strftime('%d/%m/%Y')}")
        if not df_despesas.empty:
            print(f" SAÍDAS DE CAIXA: Usando despesas provisionadas e projetando para o 4º dia útil.")
        print("="*50)

        df_fluxos_detalhados = preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas)
        
        relatorio_fluxo_caixa = gerar_relatorio_d_mais(df_fluxos_detalhados, data_referencia, feriados)
        relatorio_fluxo_caixa = gerar_relatorio_d_mais(df_fluxos_detalhados, data_referencia, feriados)
        matriz_cenarios_pdd = gerar_matriz_cenarios_pdd(df_fluxos_detalhados, data_referencia, feriados)
        
        gerar_relatorio_html_final(relatorio_fluxo_caixa, df_fluxos_detalhados, matriz_cenarios_pdd, NOME_DO_FUNDO, data_referencia, CAMINHO_DO_LOGO, NOME_ARQUIVO_SAIDA)
        
        print("\n--- PROJÇ DE FLUXO DE CAIXA ---")
        print(relatorio_fluxo_caixa)
        print("\n--- MATRIZ DE stress DE SALDO DE CAIXA ('000 R$) ---")
        matriz_print = (matriz_cenarios_pdd / 1000).applymap('{:,.0f}'.format)
        print(matriz_print)
 
        print("\ngerando relat simplificado...")

        # colunas que vao ser filtradas
        colunas_simples = [
            "Data",
            "Saldo Final<br><span class='unit'>('000 R$)</span>",
            "Índice Liq.<br><span class='unit'>(x)</span>",
            "Inadimplência B.E.<br><span class='unit'>(%)</span>",
            "Total Disp.<br><span class='unit'>('000 R$)</span>",
            "Total Nec.<br><span class='unit'>('000 R$)</span>"
        ]

        
        relatorio_simplificado = relatorio_fluxo_caixa[colunas_simples] # filtragem

        gerar_relatorio_html_simples(relatorio_simplificado, NOME_DO_FUNDO, data_referencia, NOME_ARQUIVO_SAIDA_SIMPLES)

    except FileNotFoundError as e:
        print(f"\nERRO CRÍTICO: {e}")
    except Exception as e:
        print(f"\nOcorreu um erro inesperado: {e}")

Sincronizando arquivos de Carteira e Despesas...
combinação sincronizada:
  - file de Carteira: carteira_52203615000119_20250930.json (Data: 30/09/2025)
  - Base de Despesas...: Relatório gerado em 30/09/2025

Carregando dados do novo estoque...
Usando o conjunto de estoque da data: 30.09.25
Corrigindo a PDD para base nominal...
Dados do novo estoque consolidados: 1724011 linhas.

 <DEBUG> FONTES DE DADOS UTILIZADAS 
1. PONTO DE PARTIDA (D+0): 30/09/2025
 ENTRADAS DE CAIXA: Usando conjunto de estoque gerado em 30/09/2025
 SAÍDAS DE CAIXA: Usando despesas provisionadas e projetando para o 4º dia útil.
